i use airflow

In [12]:
import pandas as pd
from sqlalchemy import create_engine

db_params = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",  # Change to your database host
    "port": "5432"       # Change to your database port
}
db_url = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"
engine = create_engine(db_url)

# Q1

In [13]:
df = pd.read_sql_query(sql="select * from public.green_tripdata", con=engine)

In [14]:
df.shape
# (266855, 20)

(266855, 20)

# Q2

In [15]:
df = pd.read_sql_query(sql=f"""
SELECT * 
FROM public.green_tripdata
WHERE passenger_count != 0 AND trip_distance != 0
                       """, con=engine)

In [16]:
#  with pandas
# df = df[(df['passenger_count'] != 0) & (df['trip_distance'] != 0)]
df.shape

(139370, 20)

# Q3

In [17]:
df['lpep_pickup_date'] = df['lpep_pickup_datetime'].dt.date

In [18]:
df['lpep_pickup_date'].head()

0    2020-10-01
1    2020-10-01
2    2020-10-01
3    2020-10-01
4    2020-10-01
Name: lpep_pickup_date, dtype: object

# Q4

In [19]:
df.VendorID.unique()

array([2, 1])

# Q5

4 col need to be renamed

In [20]:
columns = {'VendorID': 'vendor_id', 'RatecodeID': 'ratecode_id', 'PULocationID': 'pu_location_id', 'DOLocationID': 'do_location_id' }

# Create a mapping of old column names to new column names in snake_case
# column_mapping = {col: col.lower().replace(' ', '_') for col in columns}

# Rename columns using the mapping
df.rename(columns=columns)
        #   inplace=True)

,vendor_id,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,ratecode_id,pu_location_id,do_location_id,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,lpep_pickup_date
0,2,2020-10-01 00:31:19,2020-10-01 00:34:55,N,1,7,7,1,0.79,5,...,0.5,1.58,0,,0.3,7.88,1,1,0,2020-10-01
1,2,2020-10-01 00:42:12,2020-10-01 00:43:51,N,1,179,7,1,0.50,4,...,0.5,0.00,0,,0.3,5.30,2,1,0,2020-10-01
2,2,2020-10-01 00:53:09,2020-10-01 00:55:39,N,1,179,223,1,0.60,4,...,0.5,1.06,0,,0.3,6.36,1,1,0,2020-10-01
3,1,2020-10-01 00:12:29,2020-10-01 00:20:08,N,1,134,216,2,4.40,13,...,0.5,0.00,0,,0.3,14.80,2,1,0,2020-10-01
4,1,2020-10-01 00:32:38,2020-10-01 00:43:02,N,1,82,7,1,2.90,10,...,0.5,0.00,0,,0.3,11.80,2,1,0,2020-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139365,2,2020-12-31 23:08:35,2020-12-31 23:13:00,N,1,41,74,1,0.79,5,...,0.5,0.00,0,,0.3,6.30,2,1,0,2020-12-31
139366,2,2020-12-31 23:11:04,2020-12-31 23:20:08,N,5,41,116,2,2.31,12,...,0.0,0.00,0,,0.3,12.30,2,2,0,2020-12-31
139367,2,2020-12-31 23:24:33,2020-12-31 23:40:24,N,5,116,119,1,5.73,20,...,0.0,0.00,0,,0.3,20.30,2,2,0,2020-12-31
139368,2,2020-12-31 23:13:59,2020-12-31 23:16:20,N,1,42,42,1,0.31,3,...,0.5,0.00,0,,0.3,4.80,2,1,0,2020-12-31


# Q6

In [21]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/home/amal/projects/credentials/ichsanul-dev-cc6f799c9121.json', scopes=['https://www.googleapis.com/auth/cloud-platform']
)

In [22]:
import pandas as pd
from google.cloud import bigquery
from google.cloud import storage
import pyarrow.parquet as pq

# Replace with your own GCP credentials
project_id = "ichsanul-dev"
dataset_id = "de_zoomcamp"
table_id = "green_taxi"

# Assume 'df' is your pandas DataFrame

# Write to BigQuery
def write_to_bigquery(df, project_id, dataset_id, table_id):
    client = bigquery.Client(credentials=credentials, project=project_id)
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # Replace table if exists
    )

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()

    print(f'Data written to BigQuery table: {project_id}.{dataset_id}.{table_id}')

In [23]:
# # Write to BigQuery
# write_to_bigquery(df, project_id, dataset_id, table_id)

In [24]:
import pyarrow as pa
import pyarrow.parquet as pq

In [25]:
# table = pa.Table.from_pandas(df)

# # Write the PyArrow Table to a partitioned Parquet file in GCP
# pq.write_to_dataset(
#     table,
#     root_path=f"gs://dezoomcamp_ichsanuldev/green",
#     partition_cols=['lpep_pickup_date'],
#     filesystem="gs",
#     use_legacy_dataset=False,
#     # credentials=credentials
# )

In [26]:
len(df.lpep_pickup_date.unique())

95